## Load Data

In [1]:
from src.utils import load_replay_buffer_and_kernel

FOLDER_NAME: str="data/local/experiment/circle_rotation"
TASK_NAMES=["sac_circle_rotation_task_0", "sac_circle_rotation_task_1", "sac_circle_rotation_task_2", "sac_circle_rotation_task_3"]

LOAD_WHAT:str="next_observations"
KERNEL_DIM=1
N_SAMPLES=50_000

tasks_ps, tasks_frameestimators=[], []
for task_name in TASK_NAMES:
    ps, frameestimator = load_replay_buffer_and_kernel(task_name, LOAD_WHAT, KERNEL_DIM, N_SAMPLES, FOLDER_NAME)
    tasks_ps.append(ps)
    tasks_frameestimators.append(frameestimator)

2025-07-16 10:10:14.513489: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/paulnitschke/Desktop/projects/geo_meta_rl/src/utils.py:100: UserWarning: Replay buffer contains more samples than selected.
  warnings.warn("Replay buffer contains more samples than selected.")


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_0_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_1_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_2_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


Loaded next_observations from data/local/experiment/circle_rotation/sac_circle_rotation_task_3_replay_buffer.pkl with shape torch.Size([100000, 2])


INFO:root:Setup kernel frame evaluation.


In [8]:
import wandb
%load_ext autoreload
%autoreload 2

import torch
from src.learning.symmetry.hereditary_geometry_discovery import HereditaryGeometryDiscovery
from src.utils import Affine2D


train_goal_locations=[
    {'goal': torch.tensor([-0.70506063,  0.70914702])},
 {'goal': torch.tensor([ 0.95243384, -0.30474544])},
 {'goal': torch.tensor([-0.11289421, -0.99360701])},
 {'goal': torch.tensor([-0.81394263, -0.58094525])}]

SEED=42
LEARN_LEFT_ACTIONS=True
LEARN_GENERATOR=True
LEARN_ENCODER_DECODER=True
USE_ORACLE_ROTATION_KERNEL=True
N_STEPS=150_000
BATCH_SIZE=128
BANDWIDTH=None
LEARNING_RATE_LEFT_ACTIONS=0.00035
LEARNING_RATE_GENERATOR=0.00035
lr_chart=0.00015
update_chart_every_n_steps=150
N_STEPS_PRETRAIN_GEOMETRY=10_000

ENCODER=Affine2D(input_dim=2, output_dim=2)
DECODER=Affine2D(input_dim=2, output_dim=2)
ORACLE_GENERATOR=torch.tensor([[0, -1], [1,0]], dtype=torch.float32, requires_grad=False).unsqueeze(0) if not LEARN_GENERATOR else None

WAND_PROJECT_NAME="circle_hereditary_geometry_discovery"

wandb.init(project=WAND_PROJECT_NAME, name=f"lr_chart:{lr_chart}_update_n:{update_chart_every_n_steps}",config={
    "n_steps": N_STEPS,
    "batch_size": BATCH_SIZE,
    "kernel_dim": KERNEL_DIM,
    "bandwidth": BANDWIDTH,
    "learn_encoder_decoder": LEARN_ENCODER_DECODER,
    "learn_left_actions": LEARN_LEFT_ACTIONS,
    "learn_generator": LEARN_GENERATOR,
    "seed": SEED,
    "use_oracle_rotation_kernel": USE_ORACLE_ROTATION_KERNEL,
    "learning_rate_left_actions": LEARNING_RATE_LEFT_ACTIONS,
    "learning_rate_generator": LEARNING_RATE_GENERATOR,
    "learning_rate_encoder": lr_chart,
    "learning_rate_decoder": lr_chart,
    "update_chart_every_n_steps": update_chart_every_n_steps,
    "n_steps_pretrain_geometry": N_STEPS_PRETRAIN_GEOMETRY,
}, reinit=True)

her_geo_dis=HereditaryGeometryDiscovery(tasks_ps=tasks_ps,
                                        tasks_frameestimators=tasks_frameestimators, 
                                        kernel_dim=KERNEL_DIM, 
                                        batch_size=BATCH_SIZE, 
                                        seed=SEED, 
                                        bandwidth=BANDWIDTH,
                                        log_wandb=True,
                                        learn_encoder_decoder=LEARN_ENCODER_DECODER,
                                        use_oracle_rotation_kernel=USE_ORACLE_ROTATION_KERNEL,
                                        task_specifications=train_goal_locations,
                                        learn_left_actions=LEARN_LEFT_ACTIONS,
                                        learn_generator=LEARN_GENERATOR,
                                        oracle_generator=ORACLE_GENERATOR,
                                        n_steps_pretrain_geometry=N_STEPS_PRETRAIN_GEOMETRY,
                                        update_chart_every_n_steps=update_chart_every_n_steps,
                                        learning_rate_left_actions=LEARNING_RATE_LEFT_ACTIONS,
                                        learning_rate_generator=LEARNING_RATE_GENERATOR,
                                        learning_rate_encoder=lr_chart,
                                        learning_rate_decoder=lr_chart,
                                        encoder=ENCODER,
                                        decoder=DECODER)
her_geo_dis.optimize(n_steps=N_STEPS)
wandb.finish()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO:root:Fitting left-actions: True
INFO:root:Fitting generator: True
INFO:root:Fitting encoder and decoder: True
INFO:root:Fitting log-linear regressors to initialize left actions.
INFO:root:Finished fitting log-linear regressors to initialize left actions.
Hereditary Symmetry Discovery:   0%|          | 0/150000 [00:00<?, ?it/s]/Users/paulnitschke/Desktop/projects/geo_meta_rl/src/learning/symmetry/hereditary_geometry_discovery.py:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  goals = torch.stack([torch.tensor(self.task_specifications[i]['goal']) for i in self.task_idxs])
Left-Action Loss: 0.003 | Task Losses: [0.    0.007 0.002] | Task Losses (reg): 1.0470000505447388 | Generator Span Loss: 0.005 | Symmetry Loss: 0.005 | Reconstruction Loss: 0.0:   1%|          | 927/150000 [42:48<104:45:32,  2.53s/it]INFO:root:Reached maximum nu

init/identity_decoder,█▆▅▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
init/identity_encoder,█▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
init/log_left_actions,▄▄▂▂▁▁▁▁▁▁▁▁▁█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁
train/generator,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/left_actions/mean,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/regularizers/left_actions/lasso,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/regularizers/symmetry,▁▁▁▁▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█
train/symmetry/reconstruction,▁▁█▄▃▂▂▂▃▄▆▃▃▅▃▃▃▃▄▄▃▂▂▃▂▃▄▄▂▂▂▃▂▂▃▃▃▂▂▃
train/symmetry/span,█▅▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
init/identity_decoder,0.0
init/identity_encoder,0.0


In [8]:
dir="data/local/experiment/circle_rotation"
out=torch.load(dir+"/2025-07-21_16-33-47/hereditary_geometry_discovery.pt")

In [9]:
out

{'log_lgs': tensor([[[-2.3501e-04,  2.6971e+00],
          [-2.6365e+00,  2.8701e-06]],
 
         [[-7.3551e-03, -2.2611e+00],
          [ 2.2085e+00, -8.9378e-03]],
 
         [[ 1.9633e-03, -1.4098e+00],
          [ 1.3773e+00,  8.1340e-04]]]),
 'generator': tensor([[[-1.7899e-10,  3.2508e-06],
          [-3.1772e-06,  4.0491e-10]]]),
 'encoder_state_dict': OrderedDict([('linear.weight',
               tensor([[1.0128e+00, 3.4381e-04],
                       [2.4784e-04, 9.8942e-01]])),
              ('linear.bias', tensor([-0.0026,  0.0008]))]),
 'decoder_state_dict': OrderedDict([('linear.weight',
               tensor([[ 9.8729e-01, -3.9354e-04],
                       [-2.4285e-04,  1.0107e+00]])),
              ('linear.bias', tensor([ 0.0026, -0.0008]))]),
 'losses': {'left_actions': [tensor([0]),
   array(0.52724344, dtype=float32),
   array(0.5126172, dtype=float32),
   array(0.48232636, dtype=float32),
   array(0.49275506, dtype=float32),
   array(0.50591445, dtype=float32)

In [4]:
her_geo_dis.generator

Parameter containing:
tensor([[[-6.0437e-06, -5.4854e-02],
         [ 5.4203e-02,  3.0285e-05]]], requires_grad=True)

In [5]:
her_geo_dis.lgs

tensor([[[-0.8919,  0.4562],
         [-0.4475, -0.8919]],

        [[-0.6130, -0.7982],
         [ 0.7808, -0.6126]],

        [[ 0.1772, -0.9955],
         [ 0.9743,  0.1775]]], grad_fn=<LinalgMatrixExpBackward0>)

In [6]:
for param in her_geo_dis.encoder.parameters():
    print(param.data)

tensor([[1.0099e+00, 2.3543e-04],
        [1.7655e-04, 9.8977e-01]])
tensor([-0.0041, -0.0023])


In [7]:
for param in her_geo_dis.decoder.parameters():
    print(param.data)

tensor([[ 9.8997e-01, -5.9011e-04],
        [-4.0776e-04,  1.0104e+00]])
tensor([0.0041, 0.0022])
